In [ ]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def get_outcome(df):
    df['y'] = 0
    df.loc[
       (df['MIHx']==1.0) |
       (df['strokeHx']==1.0) |
       (df['CHDHx']==1.0)|
       (df['CVDHx']==1.0),
       'y'] = 1
    return df

In [ ]:
def fillna_cat(df,cat_feat):
    for feat in cat_feat:
        df[feat].fillna(df[feat].mode()[0], inplace=True)
    return df


In [ ]:
def fillna_cont(df,cont_feat):
    df= df.fillna((df[cont_feat].mean()))
    return df

In [ ]:
def process_binary(df,bin_feat):
    gender_mapping = {'Female':0,'Male':1}
    df = df.assign(gender = df['sex'].map(gender_mapping))
    return df

In [ ]:
def dichotomize_nb(df,nb_feat):
    for feat in nb_feat:
        median_feat = np.nanmedian(df[feat])
        df[feat] = [1 if x >= median_feat else 0 for x in df[feat].tolist()]
        df[feat] = pd.to_numeric(data[feat])
    return df

In [ ]:
def standardize(df,con_index):
    scaler = StandardScaler()
    scaler.fit(df[con_index],) 
    df[con_index] = scaler.transform(df[con_index], copy = True)
    return df

In [ ]:
def process(df,bin_feat,cont_feat,cat_feat,nb_feat,consider_all_exams = False):
    df = get_outcome(df)
    df = process_binary(df,bin_feat)
    df = fillna_cat(df,cat_feat)
    df = fillna_cont(df,cont_feat)
    df = dichotomize_nb(df,nb_feat)
#     df = standardize(df,cont_feat)

    if consider_all_exams:
        data = df
    else:
        data = df[df['visit']==1]
    
    return data
    

In [ ]:
# filename_ = "../../CVD_data/JHS/common_data_jhs.csv"
filename_ = "../../CVD_data/JHS/jhs_data.csv"

data = pd.read_csv(filename_)
data[data.columns[~data.columns.isin(['VisitDate'])]]
data

In [ ]:
bin_feat = ['sex']
nb_feat = ['nbSESanascore',
           'nbProblems',
           'nbCohesion',
           'nbViolence',
           'nbK3FavorFoodstore',
           'nbK3paFacilities']
cont_feat = ['sportIndex','hyIndex','activeIndex', #physical activity factors
             'darkgrnVeg','eggs','fish','idealHealthNutrition', #diet factors
             'age', 'sbp', 'hdl', 'totchol' #individual health factors 
            ]

cat_feat = ['idealHealthPA', #physical activity factors
            'idealHealthNutrition', #diet factors
            'currentSmoker','Diabetes', #behaviors and conditions
            'Income' #income information
           ]


In [ ]:
data_processed = process(data,bin_feat,cont_feat,cat_feat,nb_feat,consider_all_exams = False)

In [ ]:
data_processed

In [ ]:
data_processed.to_csv("../../CVD_data/JHS/processed_visit1_jhs_data.csv",index=False)